### What is groupby()?
Think of `.groupby()` as splitting your data into groups based on a column, then applying some operation (like sum, mean, count, min, max) to each group

In [1]:
import pandas as pd

In [2]:
data = {
    "Team": ["A", "A", "B", "B", "C", "C", "C"],
    "Points": [10, 15, 20, 25, 5, 10, 15],
    "Assists": [5, 7, 10, 5, 2, 3, 4]
}

df = pd.DataFrame(data)

In [3]:
df

,Team,Points,Assists
0,A,10,5
1,A,15,7
2,B,20,10
3,B,25,5
4,C,5,2
5,C,10,3
6,C,15,4


#### Single aggregations

In [4]:
# What's the total points per team?
df.groupby('Team').groups

{'A': [0, 1], 'B': [2, 3], 'C': [4, 5, 6]}

In [5]:
total_points_by_team = df.groupby('Team').agg({'Points': 'sum'})
total_points_by_team

,Points
Team,
A,25
B,45
C,30


In [6]:
total_points_by_team.columns, total_points_by_team.index

(Index(['Points'], dtype='object'),
 Index(['A', 'B', 'C'], dtype='object', name='Team'))

In [7]:
# Total points for Team A
total_points_by_team[total_points_by_team.index == 'A']

,Points
Team,
A,25


#### Multiple aggregations

In [8]:
df.head()

,Team,Points,Assists
0,A,10,5
1,A,15,7
2,B,20,10
3,B,25,5
4,C,5,2


In [9]:
# What's the total and average points for each team
total_avg_by_team = df.groupby('Team').agg({'Points': ['sum', 'mean']})
total_avg_by_team

Points      
        sum  mean
Team             
A        25  12.5
B        45  22.5
C        30  10.0

In [10]:
total_avg_by_team[total_avg_by_team.index == "A"]

Points      
        sum  mean
Team             
A        25  12.5

In [11]:
total_avg_by_team[total_avg_by_team.index == "A"].columns

MultiIndex([('Points',  'sum'),
            ('Points', 'mean')],
           )

In [12]:
total_avg_by_team[total_avg_by_team.index == "A"][("Points", "sum")]

Team
A    25
Name: (Points, sum), dtype: int64

### Multiple functions on multiple columns

In [13]:
df.head()

,Team,Points,Assists
0,A,10,5
1,A,15,7
2,B,20,10
3,B,25,5
4,C,5,2


In [14]:
# What's the total Points and Assists for each team
df.groupby('Team').agg({'Points': 'sum', 'Assists': 'sum'})

,Points,Assists
Team,,
A,25,12
B,45,15
C,30,9


In [15]:
# What's the total and average Points and Assists for each team
df.groupby('Team').agg({'Points': ['sum', 'mean'], 'Assists': ['sum', 'mean']})

Points       Assists     
        sum  mean     sum mean
Team                          
A        25  12.5      12  6.0
B        45  22.5      15  7.5
C        30  10.0       9  3.0

## Grouping by multiple columns

You are given a dataset of household listings with the following columns:

- host_id: unique identifier for the host
- host_name: name of the host
- is_superhost: indicator of whether the host is a superhost (t or f)
- neighbourhood: the city or neighbourhood of the listing
- price: the price of the listing

Find the neighbourhood with the smallest absolute difference between the median prices of listings from superhosts and non-superhosts.

In [18]:
listings = {
    "host_id": list(range(1, 21)),
    "host_name": [f"Host{i}" for i in range(1, 21)],
    "is_superhost": [
        "t", "f", "t", "t", "t", "f", "t", "t", "t", "f",
        "t", "t", "t", "t", "f", "t", "f", "f", "f", "t"
    ],
    "neighbourhood": [
        "CityB", "CityA", "CityB", "CityB", "CityB", "CityB", "CityA", "CityA",
        "CityB", "CityB", "CityA", "CityA", "CityA", "CityC", "CityC", "CityC",
        "CityB", "CityC", "CityB", "CityB"
    ],
    "price": [
        224, 239, 100, 157, 104, 293, 113, 298, 180, 278,
        100, 184, 70, 122, 216, 67, 181, 138, 109, 63
    ]
}

df = pd.DataFrame(listings)

In [19]:
df.head()

,host_id,host_name,is_superhost,neighbourhood,price
0,1,Host1,t,CityB,224
1,2,Host2,f,CityA,239
2,3,Host3,t,CityB,100
3,4,Host4,t,CityB,157
4,5,Host5,t,CityB,104


In [21]:
grouped = df.groupby(['neighbourhood', 'is_superhost']).agg({"price": "median"})
grouped

price
neighbourhood is_superhost       
CityA         f             239.0
              t             113.0
CityB         f             229.5
              t             130.5
CityC         f             177.0
              t              94.5

In [23]:
unstacked_grouped = grouped.unstack(level="is_superhost")
unstacked_grouped

price       
is_superhost       f      t
neighbourhood              
CityA          239.0  113.0
CityB          229.5  130.5
CityC          177.0   94.5

In [25]:
unstacked_grouped.columns

MultiIndex([('price', 'f'),
            ('price', 't')],
           names=[None, 'is_superhost'])

In [26]:
unstacked_grouped[('price', 'f')]

neighbourhood
CityA    239.0
CityB    229.5
CityC    177.0
Name: (price, f), dtype: float64

In [28]:
unstacked_grouped["diff"] = (unstacked_grouped[('price', 't')] - unstacked_grouped[('price', 'f')]).abs()
unstacked_grouped

price          diff
is_superhost       f      t       
neighbourhood                     
CityA          239.0  113.0  126.0
CityB          229.5  130.5   99.0
CityC          177.0   94.5   82.5

In [30]:
unstacked_grouped["diff"].idxmin()

'CityC'

In [32]:
unstacked_grouped.loc['CityC']

       is_superhost
price  f               177.0
       t                94.5
diff                    82.5
Name: CityC, dtype: float64